### Imports and Setup

In [33]:
%cd /home/ubuntu/eda/data
from osgeo import gdal, gdal_array, osr, ogr
import numpy as np
import rasterio
from rasterio import mask
import pyproj
from affine import Affine
import pandas as pd
from shapely.geometry import Point
from geopandas import GeoDataFrame
import geopandas as gpd
from rasterio.features import shapes
import xarray
import matplotlib.pyplot as plt
import rtree
import shapely
import warnings
import time
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore')

/home/ubuntu/eda/data


### Load census block and census tract data

In [34]:
#census blocks
cb_df = gpd.read_file('./shapefiles/nycb2020_22b/nycb2020.shp')
cb_df.columns = cb_df.columns.str.lower()

#census tracts
ct_df = gpd.read_file('./shapefiles/nyct2020_22b/nyct2020.shp')
ct_df.columns = ct_df.columns.str.lower()

### Create feature generation functions

In [35]:
def gen_lc_features(df, filepath, num_classes = 9, prefix = 'land_cvr_', default_val = 0, ct_level = False):
    
    if ct_level:
        suffix = '_ct'
    else:
        suffix = ''
        
    lc_data = rasterio.open(filepath)
    feat_names = [prefix+str(i)+suffix for i in range(1,num_classes)] 
    # if rebase:
    #     rb_feat_names = [prefix + str(i) + '_rb' for i in range(1,num_classes)]
    for feat in feat_names:
        df[feat] = default_val
    
    for i in range(len(df)):
        filt_raster, _ = mask.mask(lc_data, [df['geometry'][i]], crop = True)
        lc_proportion = np.histogram(filt_raster, bins = [i for i in range(num_classes + 1)])[0]/(filt_raster.shape[1]*filt_raster.shape[2])
        for j in range(1,len(lc_proportion)):
            var_name = prefix + str(j) + suffix
            df.loc[i, var_name] = round(lc_proportion[j]/(1-lc_proportion[0]),6)
    if ct_level:
        return df[['ct2020','ntaname']+feat_names]
    else:
        return df

In [53]:
def gen_elev_features(df, filepath, prefix = 'elev_', ct_level = False, nodata_val = 999):
    
    if ct_level:
        suffix = '_ct'
    else:
        suffix = ''
    
    elev_data = rasterio.open(filepath)
    feat_names = [prefix + i + suffix for i in ['mean','min','max','q1','q3','var']] 
    for feat in feat_names:
        df[feat] = nodata_val
    
    for i in range(len(df)):
        filt_raster, _ = mask.mask(elev_data, [df['geometry'][i]], crop = True, nodata = nodata_val)
        filt_raster = np.where(filt_raster == nodata_val, np.nan, filt_raster)
        df.loc[i, prefix + 'mean' + suffix] = round(np.nanmean(filt_raster),1)
        df.loc[i, prefix + 'var' + suffix] = round(np.nanvar(filt_raster),1)
        df.loc[i, prefix + 'min' + suffix], df.loc[i, prefix + 'q1' + suffix], df.loc[i, prefix + 'q3' + suffix], df.loc[i, prefix + 'max' + suffix] = np.nanpercentile(filt_raster, [0,25,75,100])
    if ct_level:
        return df[['ct2020']+feat_names]
    else:
        return df

In [54]:
def gen_drain_features(df, filepath):
    
    drain_data = gpd.read_file(filepath)
    drain_data.columns = drain_data.columns.str.lower()

    drain_data = drain_data.sjoin(df, how = 'inner', predicate = 'within')[['unitid','bctcb2020']]                                  
    df = df.merge(drain_data.groupby('bctcb2020')['unitid'].nunique(), how = 'left', on = 'bctcb2020').rename(columns = {'unitid':'catch_basin_count'})
    df['catch_basin_density'] = df['catch_basin_count']/df['shape_area']
    
    return df

In [55]:
def gen_subway_features(df, filepath):
    
    sub_data = gpd.read_file(filepath)
    sub_data.columns = sub_data.columns.str.lower()

    sub_data = sub_data.sjoin(df, how = 'inner', predicate = 'within')[['objectid','bctcb2020']]                                  
    df = df.merge(sub_data.groupby('bctcb2020')['objectid'].nunique(), how = 'left', on = 'bctcb2020').rename(columns = {'objectid':'sub_entr_count'})
    
    return df

In [56]:
def gen_ret_wall_features(df, filepath):
    
    rw_data = gpd.read_file(filepath)
    rw_data.columns = rw_data.columns.str.lower()
    
    rw_data = df.overlay(rw_data, how = 'intersection', keep_geom_type = False)[['bctcb2020','shape_leng_2','geometry']]
    
    #explode to split any multipart geometries
    rw_data = rw_data.explode(ignore_index = True)
    
    length = []
    avg_rw_elev = []
    cb_list = []
    for cb in rw_data.bctcb2020.unique():
        cb_list.append(cb)
        elev_list = []
        subset = rw_data[rw_data.bctcb2020 == cb]
        length.append(subset['shape_leng_2'].sum())
        for i in range(0,len(subset)):
            for j in list(subset.iloc[i].geometry.coords):
                 elev_list.append(j[2])
        avg_rw_elev.append(np.mean(elev_list))
    
    rw_agg = pd.DataFrame(zip(cb_list, length, avg_rw_elev),columns  = ['bctcb2020','rw_length','rw_avg_elev'])
    df = df.merge(rw_agg, how = 'left', on = 'bctcb2020')
    
    return df

In [57]:
def gen_hydro_features(df, filepath, hydro_dict):
    
    h_data = gpd.read_file(filepath)
    h_data.columns = h_data.columns.str.lower()
    
    h_data = df.overlay(h_data, how = 'intersection', keep_geom_type = False)
    
    for key in hydro_dict.keys():
        df[hydro_dict[key]] = np.where(df.bctcb2020.isin(h_data[h_data.feat_code == key].bctcb2020),1,0)
    
    return df

In [58]:
lc_path = './raster_data/NYC_2017_LiDAR_LandCover.img'
elev_path = './elevation/DEM_LiDAR_1ft_2010_Improved_NYC_int.tif'
dr_path = './DEPCatchbasins/DEPCATCHBASINS.shp'
se_path = './doitt_subway_entrances/DOITT_SUBWAY_ENTRANCE_04JAN2017.shp'
rw_path = './retaining_wall/RETAININGWALL.shp'
h_path = './hydro/HYDROGRAPHY.shp'

hydro_dict = {2600:'lake_res_ind',
              2610:'pond_ind',
              2620:'river_ind',
              2630:'stream_ind',
              2640:'wl_marsh_ind',
              2650:'beach_shore_ind',
              2660:'bay_ocean_ind'}

### Borough-Level Dataset Pipeline (WIP)

In [60]:
def gen_boro_dataset(cb_df, ct_df, name, name_abv):
    start_ = time.time()
    #land cover
    df = gen_lc_features(cb_df[cb_df.boroname == name].reset_index(drop = True), lc_path)
    print(f'CB Land cover features complete: {round((time.time()-start_)/60,1)} total time elapsed')
    df_c = gen_lc_features(ct_df[ct_df.boroname == name].reset_index(drop = True), lc_path, ct_level = True)
    print(f'CT Land cover features complete: {round((time.time()-start_)/60,1)} total time elapsed')
    df = df.merge(df_c, on = 'ct2020')

    #elevation
    df = gen_elev_features(df, elev_path)
    print(f'CB elevation features complete: {round((time.time()-start_)/60,1)} total time elapsed')
    df_c = gen_elev_features(ct_df[ct_df.boroname == name].reset_index(drop = True), elev_path, ct_level = True)
    print(f'CT elevation features complete: {round((time.time()-start_)/60,1)} total time elapsed')
    df = df.merge(df_c, on = 'ct2020')
    
    #drainage/water
    df = gen_drain_features(df, dr_path)
    print(f'Catch basin features complete: {round((time.time()-start_)/60,1)} total time elapsed')
    df = gen_subway_features(df, se_path)
    print(f'Subway features complete: {round((time.time()-start_)/60,1)} total time elapsed')
    df = gen_ret_wall_features(df, rw_path)
    print(f'Retaining wall features complete: {round((time.time()-start_)/60,1)} total time elapsed')
    df = gen_hydro_features(df, h_path, hydro_dict)
    print(f'Hydrography features complete: {round((time.time()-start_)/60,1)} total time elapsed')
    df.to_csv(f'{name_abv}_data.csv')
    
    return df

### Generate full dataset

##### Manhattan

In [61]:
# test = cb_df[cb_df.boroname == 'Manhattan']
# elev_data = rasterio.open(elev_path)
# nodata_val = -9999
# for i in range(len(test)):
#     filt_raster, _ = mask.mask(elev_data, [test['geometry'][i]], crop = True, nodata = 999)
#     print(filt_raster)
#     filt_raster = np.where(filt_raster == 999, np.nan, filt_raster)
#     print(filt_raster)
#     print(np.nanmean(filt_raster))
#     print(np.nanpercentile(filt_raster, [0,25,75,100]))
#     break

In [62]:
mht = gen_boro_dataset(cb_df, ct_df, 'Manhattan', 'mht')

CB Land cover features complete: 2.9 total time elapsed
CT Land cover features complete: 5.5 total time elapsed
CB elevation features complete: 7.9 total time elapsed
CT elevation features complete: 11.0 total time elapsed
Catch basin features complete: 11.5 total time elapsed
Subway features complete: 11.6 total time elapsed
Retaining wall features complete: 11.6 total time elapsed
Hydrography features complete: 11.7 total time elapsed


In [63]:
!aws s3 cp mht_data.csv s3://w210-flood-risk/modeling_data/mht_data.csv --acl public-read

upload: ./mht_data.csv to s3://w210-flood-risk/modeling_data/mht_data.csv


##### Brooklyn

In [64]:
bk = gen_boro_dataset(cb_df, ct_df, 'Brooklyn', 'bk')

CB Land cover features complete: 7.7 total time elapsed
CT Land cover features complete: 14.6 total time elapsed
CT elevation features complete: 28.3 total time elapsed
Catch basin features complete: 28.8 total time elapsed
Subway features complete: 28.8 total time elapsed
Retaining wall features complete: 28.9 total time elapsed
Hydrography features complete: 29.0 total time elapsed


In [65]:
!aws s3 cp bk_data.csv s3://w210-flood-risk/modeling_data/bk_data.csv --acl public-read

upload: ./bk_data.csv to s3://w210-flood-risk/modeling_data/bk_data.csv


##### Queens

In [66]:
qns = gen_boro_dataset(cb_df, ct_df, 'Queens', 'qns')

CB Land cover features complete: 12.2 total time elapsed
CT Land cover features complete: 23.8 total time elapsed
CB elevation features complete: 36.1 total time elapsed
CT elevation features complete: 46.5 total time elapsed
Catch basin features complete: 47.0 total time elapsed
Subway features complete: 47.1 total time elapsed
Retaining wall features complete: 47.2 total time elapsed
Hydrography features complete: 47.3 total time elapsed


In [67]:
!aws s3 cp qns_data.csv s3://w210-flood-risk/modeling_data/qns_data.csv --acl public-read

upload: ./qns_data.csv to s3://w210-flood-risk/modeling_data/qns_data.csv


##### Bronx

In [68]:
bx = gen_boro_dataset(cb_df, ct_df, 'Bronx', 'bx')

CB Land cover features complete: 5.4 total time elapsed
CT Land cover features complete: 10.0 total time elapsed
CB elevation features complete: 13.5 total time elapsed
CT elevation features complete: 17.3 total time elapsed
Catch basin features complete: 17.8 total time elapsed
Subway features complete: 17.9 total time elapsed
Retaining wall features complete: 17.9 total time elapsed
Hydrography features complete: 18.0 total time elapsed


In [69]:
!aws s3 cp bx_data.csv s3://w210-flood-risk/modeling_data/bx_data.csv --acl public-read

upload: ./bx_data.csv to s3://w210-flood-risk/modeling_data/bx_data.csv


##### Staten Island

In [70]:
si = gen_boro_dataset(cb_df, ct_df, 'Staten Island', 'si')

CB Land cover features complete: 7.4 total time elapsed
CT Land cover features complete: 13.3 total time elapsed
CB elevation features complete: 18.6 total time elapsed
CT elevation features complete: 23.9 total time elapsed
Catch basin features complete: 24.3 total time elapsed
Subway features complete: 24.4 total time elapsed
Retaining wall features complete: 24.4 total time elapsed
Hydrography features complete: 24.5 total time elapsed


In [71]:
!aws s3 cp si_data.csv s3://w210-flood-risk/modeling_data/si_data.csv --acl public-read

upload: ./si_data.csv to s3://w210-flood-risk/modeling_data/si_data.csv


#### Combine boroughs

In [105]:
ny = pd.concat([mht,bk,qns,bx,si],ignore_index = True)
ny['elev_mean_diff'] = ny['elev_mean'] - ny['elev_mean_ct']
ny = ny.fillna(0)

In [106]:
ny.to_csv('full_ny.csv')

In [107]:
!aws s3 cp full_ny.csv s3://w210-flood-risk/modeling_data/full_ny.csv --acl public-read

upload: ./full_ny.csv to s3://w210-flood-risk/modeling_data/full_ny.csv


### Standardization

In [108]:
scale_cols = [col for col in ny.columns if ny[col].dtype not in ['object','Polygon','geometry'] and '_ind' not in col]
print(len(scale_cols))
scale_cols

36


['shape_leng',
 'shape_area',
 'land_cvr_1',
 'land_cvr_2',
 'land_cvr_3',
 'land_cvr_4',
 'land_cvr_5',
 'land_cvr_6',
 'land_cvr_7',
 'land_cvr_8',
 'land_cvr_1_ct',
 'land_cvr_2_ct',
 'land_cvr_3_ct',
 'land_cvr_4_ct',
 'land_cvr_5_ct',
 'land_cvr_6_ct',
 'land_cvr_7_ct',
 'land_cvr_8_ct',
 'elev_mean',
 'elev_min',
 'elev_max',
 'elev_q1',
 'elev_q3',
 'elev_var',
 'elev_mean_ct',
 'elev_min_ct',
 'elev_max_ct',
 'elev_q1_ct',
 'elev_q3_ct',
 'elev_var_ct',
 'catch_basin_count',
 'catch_basin_density',
 'sub_entr_count',
 'rw_length',
 'rw_avg_elev',
 'elev_mean_diff']

In [109]:
scaler = StandardScaler()
ny_scale = scaler.fit_transform(ny[scale_cols])
ny_scale.shape

(37589, 36)

In [110]:
ny[scale_cols] = ny_scale

In [111]:
ny

,cb2020,borocode,boroname,ct2020,bctcb2020,geoid,shape_leng,shape_area,geometry,land_cvr_1,...,rw_length,rw_avg_elev,lake_res_ind,pond_ind,river_ind,stream_ind,wl_marsh_ind,beach_shore_ind,bay_ocean_ind,elev_mean_diff
0,1000,1,Manhattan,000100,10001001000,360610001001000,2.563287,1.028343,"POLYGON ((973172.666 194632.348, 973310.630 19...",0.572610,...,-0.109035,-0.197793,0,0,1,0,0,0,1,-0.103761
1,1001,1,Manhattan,000100,10001001001,360610001001001,1.328588,0.436533,"POLYGON ((972081.788 190733.467, 972184.766 19...",0.232433,...,-0.109035,-0.197793,0,0,0,0,0,1,1,0.168386
2,1000,1,Manhattan,000201,10002011000,360610002011000,-0.234124,-0.099464,"POLYGON ((988376.731 199328.618, 987837.811 19...",-0.545577,...,-0.109035,-0.197793,0,0,0,0,0,0,0,0.699072
3,1001,1,Manhattan,000201,10002011001,360610002011001,-0.220366,-0.088884,"POLYGON ((988392.400 199070.298, 988285.301 19...",0.030490,...,-0.109035,-0.197793,0,0,0,0,0,0,0,0.603821
4,2000,1,Manhattan,000201,10002012000,360610002012000,0.034592,0.041155,"POLYGON ((988422.186 198807.188, 988449.289 19...",3.077870,...,-0.109035,-0.197793,0,0,0,0,0,0,0,0.318067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37584,1025,5,Staten Island,032300,50323001025,360850323001025,2.277571,0.962647,"POLYGON ((937477.858 165774.254, 937409.633 16...",1.239850,...,-0.109035,-0.197793,0,0,0,1,1,0,0,0.025509
37585,1014,5,Staten Island,032300,50323001014,360850323001014,0.241679,0.090793,"POLYGON ((938125.104 170978.846, 938101.030 17...",-0.535294,...,-0.109035,-0.197793,0,0,0,0,0,0,0,1.495101
37586,1015,5,Staten Island,032300,50323001015,360850323001015,5.924487,7.798783,"POLYGON ((938097.974 170133.145, 938095.423 17...",0.774925,...,0.591361,0.827865,0,1,0,1,1,0,0,0.127564
37587,0005,5,Staten Island,990100,59901000005,360859901000005,-0.141777,-0.100487,"POLYGON ((970217.022 145643.332, 970227.216 14...",-1.122841,...,-0.109035,-0.197793,0,0,0,0,0,0,1,-0.008510


In [112]:
ny.to_csv('ny_full_scaled.csv')

In [113]:
!aws s3 cp ny_full_scaled.csv s3://w210-flood-risk/modeling_data/ny_full_scaled.csv --acl public-read

upload: ./ny_full_scaled.csv to s3://w210-flood-risk/modeling_data/ny_full_scaled.csv
